# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### 1) What test is appropriate for this problem? Does CLT apply?
We can apply permutation tests (bootstrap).

Yes, Central limit theory applies. (if data transformed vertically, we have more values i.e, more than 30)

### 2) What are the null and alternate hypotheses?
Null hypothesis (H-zero): race does not effect the callback.

Alternate hypothesis (H-alpha): race does effect the callback.

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

### 3) Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [6]:
white_callback = w['call']
black_callback = b['call']

In [7]:
# frequentist
white_std = white_callback.std()
black_std = black_callback.std()
mean_diff = np.abs(np.mean(white_callback) - np.mean(black_callback))
print('white std:\t',white_std,'\nblack std:\t',black_std,'\nmean-diff:\t',mean_diff)
std_err = np.sqrt(white_std**2/len(white_callback) + black_std**2/len(black_callback))
print('error:\t\t',std_err)

white std:	 0.2953455150127411 
black std:	 0.24564945697784424 
mean-diff:	 0.0320328548551
error:		 0.00778490691981


In [8]:
# confidence interval
confidence_interval = [mean_diff - 1.96*std_err, mean_diff + 1.96*std_err]
print('95% confidence:',confidence_interval)

95% confidence: [0.016774437292225546, 0.047291272417895609]


In [9]:
# p-value
t, p = stats.ttest_ind(white_callback,black_callback)
print('stats:\t',t,'\np val:\t',p)

stats:	 4.11470529086 
p val:	 3.94080210313e-05


In [10]:
# boostrapping
permutation_replicates = np.empty(10000)

for i in range(10000):
    combined = np.random.permutation(np.concatenate((white_callback,black_callback)))
    white_permutation = combined[:len(white_callback)]
    black_permutation = combined[len(white_callback):]
    permutation_replicates[i] = np.abs(np.mean(white_permutation) - np.mean(black_permutation))

perm_p_val = np.sum(permutation_replicates > mean_diff)/10000
print('p val:\t',perm_p_val)

p val:	 0.0


p value is very low using frequentist, bootstrapping.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

### 4) Write a story describing the statistical significance in the context or the original problem.

Racial discrimination is pervasive throughout the world. We have investigated a data set and looked at the distribution and we do see that the racial component is prevailing when it comes to callback for job.

The p-value found via t-test and permutations are really low (means it has a statistical significance, may not be practical), so we can reject the null hypothesis. And can say that race was considered for a job callback.

### 5) Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Here race/name was a factor for a callback, it does not mean it is an important factor. We need to do a furthur investigation on the data set not only on the race/name.